In [ ]:
# bits croper

from ultralytics import YOLO
import cv2
import numpy as np
import os

model = YOLO("./model/best.pt")

def sort_boxes_xyxy(boxes):
    boxes = np.array(boxes)
    sorted_by_y = boxes[boxes[:, 1].argsort()]
    row1 = sorted_by_y[:17]
    row2 = sorted_by_y[17:]
    row1 = row1[row1[:, 0].argsort()]
    row2 = row2[row2[:, 0].argsort()]
    return np.vstack((row1, row2))


os.makedirs("./crops", exist_ok=True)

for i in range(1, 106):

    image_path = f"./images/{i}.jpg"
    results = model.predict(image_path, conf=0.45)[0]
    img = cv2.imread(image_path)

    padding = 0

    all_boxes = []
    all_confs = []

    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf.cpu().numpy())
        all_boxes.append([x1, y1, x2, y2])
        all_confs.append(conf)

    all_boxes = np.array(all_boxes)
    all_confs = np.array(all_confs)

    # pick top 34 by confidence
    keep_idx = np.argsort(-all_confs)[:34]
    boxes_34 = all_boxes[keep_idx]

    # sort into 2 rows and left→right
    sorted_boxes = sort_boxes_xyxy(boxes_34)

    # remove boxes that are too close
    filtered_boxes = []
    last_x = None
    for box in sorted_boxes:
        x1, y1, x2, y2 = box
        if last_x is None:
            filtered_boxes.append(box)
            last_x = x1
        else:
            if abs(x1 - last_x) >= 20:
                filtered_boxes.append(box)
                last_x = x1

    filtered_boxes = np.array(filtered_boxes)

    # ---- CROP AND SAVE ----
    box_id = 1
    for (x1, y1, x2, y2) in filtered_boxes:
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

        # apply padding safely
        px1 = max(0, x1 - padding)
        py1 = max(0, y1 - padding)
        px2 = min(img.shape[1], x2 + padding)
        py2 = min(img.shape[0], y2 + padding)
        margin = 19
        crop = img[py1+margin:py2-margin, px1+margin:px2-margin]

        out_path = f"./crops/{i}-{box_id}.jpg"
        cv2.imwrite(out_path, crop)
        box_id += 1

    print("done", i)


In [ ]:
# box drawing code with numbers

from ultralytics import YOLO
import cv2
import numpy as np
model = YOLO("./model/best.pt")

def sort_boxes_xyxy(boxes):
    boxes = np.array(boxes)
    sorted_by_y = boxes[boxes[:, 1].argsort()]
    row1 = sorted_by_y[:17]
    row2 = sorted_by_y[17:]
    row1 = row1[row1[:, 0].argsort()]
    row2 = row2[row2[:, 0].argsort()]
    return np.vstack((row1, row2))

a = []
for i in range(1, 106):
    image_path = f"./images/{i}.jpg"
    results = model.predict(image_path, conf=0.45)[0]
    img = cv2.imread(image_path)
    BLUE, THICKNESS, padding = (255, 0, 0), 7, 34
    all_boxes = []
    all_confs = []

    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf.cpu().numpy())
        all_boxes.append([x1, y1, x2, y2])
        all_confs.append(conf)

    all_boxes = np.array(all_boxes)
    all_confs = np.array(all_confs)
    keep_idx = np.argsort(-all_confs)[:34]
    boxes_34 = all_boxes[keep_idx]
    sorted_boxes = sort_boxes_xyxy(boxes_34)
    filtered_boxes = []
    last_x = None

    for box in sorted_boxes:
        x1, y1, x2, y2 = box

        if last_x is None:
            filtered_boxes.append(box)
            last_x = x1
        else:
            if abs(x1 - last_x) >= 20:
                filtered_boxes.append(box)
                last_x = x1

    filtered_boxes = np.array(filtered_boxes)
    box_id = 1

    
    a.append(len(filtered_boxes))
    
    for (x1, y1, x2, y2) in filtered_boxes:
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        cv2.rectangle(img, (x1 + padding, y1 + padding), (x2 - padding, y2 - padding), BLUE, THICKNESS)
        cv2.putText(img, str(box_id), (x1 + padding, y1 + padding - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, BLUE, 3)
        box_id += 1

    out_path = f"./box_with_numbers/{i}.jpg"
    cv2.imwrite(out_path, img)
    print("done", i)


In [ ]:
# image tester code

from ultralytics import YOLO
import cv2

model = YOLO("./model/best.pt")   # or "epoch_60.pt", your choice
image_path = "./images/100.jpg"
results = model.predict(image_path)[0]
img = cv2.imread(image_path)
BLUE = (255, 0, 0)
THICKNESS = 7

for box in results.boxes:
    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
    cv2.rectangle(
        img,
        (int(x1), int(y1)),
        (int(x2), int(y2)),
        BLUE,
        THICKNESS
    )

out_path = "result.jpg"
cv2.imwrite(out_path, img)
print("done")

In [ ]:
# image rotater

import os
from PIL import Image
from pillow_heif import register_heif_opener

register_heif_opener()

folder = r"D:\zz train\images"

for filename in os.listdir(folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png", ".heic")):
        img_path = os.path.join(folder, filename)
        
        try:
            img = Image.open(img_path)
            rotated = img.rotate(90, expand=True)
            rotated.save(img_path)
            print("Rotated:", img_path)

        except Exception as e:
            print("Skipping:", img_path, "| Error:", e)


In [ ]:
# images name renamer

import os
folder = f"../zz train/images"
start_number = 40
files = os.listdir(folder)
image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.heic'))]
image_files.sort()
current_number = start_number

for filename in image_files:
    old_path = os.path.join(folder, filename)
    new_name = f"{current_number}.jpg"
    new_path = os.path.join(folder, new_name)
    os.rename(old_path, new_path)
    current_number += 1

In [ ]:
# image saturation

import cv2
import numpy as np
import glob
import os

input_folder = "./crops/"
output_folder = "./crops_output/"
os.makedirs(output_folder, exist_ok=True)

lower_red1 = np.array([0, 15, 15])
upper_red1 = np.array([25, 255, 255])

lower_red2 = np.array([155, 15, 15])
upper_red2 = np.array([179, 255, 255])

kernel = np.ones((3,3), np.uint8)

for img_path in glob.glob(input_folder + "/*.jpg"):  # change to jpg if needed
    img = cv2.imread(img_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 | mask2

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    h, s, v = cv2.split(hsv)

    s = s.astype(np.float32)
    s[mask > 0] *= 4
    s = np.clip(s, 0, 255).astype(np.uint8)

    hsv_enhanced = cv2.merge([h, s, v])
    final_img = cv2.cvtColor(hsv_enhanced, cv2.COLOR_HSV2BGR)

    filename = os.path.basename(img_path)
    cv2.imwrite(os.path.join(output_folder, filename), final_img)

    print("Saved:", filename)

In [ ]:
# image saturation with thresholding(THRESH_BINARY_INV)

import cv2
import numpy as np
import glob
import os

input_folder = "./crops/"
binary_output_folder = "./crops_binary_inv/"

os.makedirs(binary_output_folder, exist_ok=True)

lower_red1 = np.array([0, 15, 15])
upper_red1 = np.array([25, 255, 255])

lower_red2 = np.array([155, 15, 15])
upper_red2 = np.array([179, 255, 255])

kernel = np.ones((13,13), np.uint8)

for img_path in glob.glob(input_folder + "/*.jpg"):
    img = cv2.imread(img_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 | mask2

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    h, s, v = cv2.split(hsv)

    s = s.astype(np.float32)
    s[mask > 0] *= 4
    s = np.clip(s, 0, 255).astype(np.uint8)

    hsv_enhanced = cv2.merge([h, s, v])
    final_img = cv2.cvtColor(hsv_enhanced, cv2.COLOR_HSV2BGR)

    filename = os.path.basename(img_path)

    gray = cv2.cvtColor(final_img, cv2.COLOR_BGR2GRAY)
    _, binary_inv = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)
    binary_inv = cv2.resize(binary_inv, (32, 32), interpolation=cv2.INTER_NEAREST)

    cv2.imwrite(os.path.join(binary_output_folder, filename), binary_inv)

    print("Saved:", filename, "and binary_inv version")

In [ ]:
# images sorter

import os
import cv2
import torch
import numpy as np
import shutil
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

class ResBlock(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(),

            nn.Conv2d(out_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c)
        )

        self.shortcut = (
            nn.Sequential(
                nn.Conv2d(in_c, out_c, 1, bias=False),
                nn.BatchNorm2d(out_c)
            )
            if in_c != out_c else nn.Identity()
        )

        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv(x)
        out += self.shortcut(x)
        return self.relu(out)

class StrongCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            ResBlock(1, 32),
            nn.MaxPool2d(2),

            ResBlock(32, 64),
            nn.MaxPool2d(2),  # becomes 8x8

            ResBlock(64, 128),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 12)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = StrongCNN().to(device)
model.load_state_dict(torch.load("./model/model_epoch_190.pt", map_location=device))
model.eval()

print("✔ Model loaded successfully")

input_folder = "./crops_brigntness_fixed_thresh"
output_root = "./sorted_digits_2"
os.makedirs(output_root, exist_ok=True)

for d in range(12):
    os.makedirs(os.path.join(output_root, str(d)), exist_ok=True)

for fname in os.listdir(input_folder):
    if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
        continue

    path = os.path.join(input_folder, fname)
    
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (32, 32))
    img = img.astype(np.float32) / 255.0

    img = torch.tensor(img).unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(img).argmax(dim=1).item()

    shutil.copy(path, os.path.join(output_root, str(pred), fname))

print("\n✔ All predictions completed and images sorted into folders.")


In [ ]:
# image generator with out box but adding noise and augmenting images

import cv2
import numpy as np
import os
import random

INPUT_DIR = "./normal data"
OUTPUT_DIR = "./augmented"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------- AUGMENTATION FUNCTIONS ----------

def random_translate(img):
    h, w = img.shape
    tx = random.randint(-7, 7)
    ty = random.randint(-7, 7)
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    return cv2.warpAffine(img, M, (w, h), borderValue=0)

def add_noise(img):
    choice = random.choice(["gaussian", "saltpepper", "none"])
    img = img.copy()

    if choice == "gaussian":
        noise = np.random.normal(0, 20, img.shape).astype(np.int16)
        img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    elif choice == "saltpepper":
        prob = 0.02
        rnd = np.random.rand(*img.shape)
        img[rnd < prob] = 0
        img[rnd > 1 - prob] = 255

    return img

def random_scale(img):
    scale = random.uniform(0.85, 1.15)
    resized = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_NEAREST)

    canvas = np.zeros_like(img)
    h, w = img.shape
    rh, rw = resized.shape

    y = (h - rh) // 2
    x = (w - rw) // 2

    if y >= 0 and x >= 0:
        canvas[y:y+rh, x:x+rw] = resized[:h-y, :w-x]
    else:
        canvas = cv2.resize(resized, (w, h), interpolation=cv2.INTER_NEAREST)

    return canvas

def random_rotate(img):
    angle = random.uniform(-10, 10)
    h, w = img.shape
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
    return cv2.warpAffine(img, M, (w, h), borderValue=0)

def random_margin_crop(img):
    # crop and pad simulated
    top = random.randint(0, 3)
    bottom = random.randint(0, 3)
    left = random.randint(0, 3)
    right = random.randint(0, 3)

    cropped = img[top:28-bottom, left:28-right]

    padded = cv2.copyMakeBorder(
        cropped,
        top, bottom, left, right,
        cv2.BORDER_CONSTANT,
        value=0
    )
    return padded

# ---- EXTRA: tiny strokes for empty class ----

def random_stray_stroke():
    img = np.zeros((28, 28), dtype=np.uint8)

    # 70% chance no stroke (empty)
    if random.random() < 0.7:
        return img

    # 30% chance: generate stroke
    length = random.randint(3, 9)
    thickness = random.randint(1, 3)
    angle = random.uniform(-40, 40)  # roughly horizontal

    # pick LEFT or RIGHT side
    if random.random() < 0.5:
        # LEFT SIDE
        x1 = random.randint(0, 5)
    else:
        # RIGHT SIDE
        x1 = random.randint(22, 27)

    y1 = random.randint(5, 22)  # avoid top/bottom

    # compute endpoint from angle + length
    dx = int(length * np.cos(np.radians(angle)))
    dy = int(length * np.sin(np.radians(angle)))

    x2 = x1 + dx
    y2 = y1 + dy

    # clamp endpoints
    x2 = np.clip(x2, 0, 27)
    y2 = np.clip(y2, 0, 27)

    cv2.line(img, (x1, y1), (x2, y2), color=255, thickness=thickness)

    return img


# ---------- MAIN PROCESS ----------

for cls in range(0, 12):  # 0–11
    in_path = os.path.join(INPUT_DIR, str(cls))
    out_path = os.path.join(OUTPUT_DIR, str(cls))
    os.makedirs(out_path, exist_ok=True)

    file_list = os.listdir(in_path)

    if cls == 11:
        # empty class special handling
        for i in range(5000):  # generate 5k empty samples
            img = random_stray_stroke()
            cv2.imwrite(os.path.join(out_path, f"{i}.png"), img)
        print("Generated empty-class augmented images.")
        continue

    # normal digit classes 0–10
    for fname in file_list:
        img = cv2.imread(os.path.join(in_path, fname), cv2.IMREAD_GRAYSCALE)

        for k in range(4):  # generate 4 augmented copies per image
            aug = img.copy()
            aug = random_translate(aug)
            aug = random_scale(aug)
            aug = random_rotate(aug)
            aug = random_margin_crop(aug)
            aug = add_noise(aug)

            save_name = fname.replace(".png", "").replace(".jpg", "")
            cv2.imwrite(os.path.join(out_path, f"{save_name}_aug{k}.png"), aug)

    print("Done class:", cls)

In [ ]:
# removing exes data

import os
import random

DATASET_DIR = "./augmented"   # change if needed
KEEP_COUNT = 7000

for cls in sorted(os.listdir(DATASET_DIR)):
    cls_path = os.path.join(DATASET_DIR, cls)
    if not os.path.isdir(cls_path):
        continue

    # list all image files
    files = [f for f in os.listdir(cls_path)
             if f.lower().endswith((".png", ".jpg", ".jpeg"))]

    print(f"Class {cls}: found {len(files)} images")

    if len(files) <= KEEP_COUNT:
        print(f"Skipping class {cls} (only {len(files)}, nothing to delete)")
        continue

    # choose 7000 random files to keep
    keep_files = set(random.sample(files, KEEP_COUNT))

    # delete everything else
    deleted = 0
    for f in files:
        if f not in keep_files:
            os.remove(os.path.join(cls_path, f))
            deleted += 1

    print(f"Class {cls}: kept {KEEP_COUNT}, deleted {deleted}")


In [ ]:
# model training

import os
import cv2
import torch
import numpy as np
import shutil
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

class ResBlock(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(),

            nn.Conv2d(out_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c)
        )

        self.shortcut = (
            nn.Sequential(
                nn.Conv2d(in_c, out_c, 1, bias=False),
                nn.BatchNorm2d(out_c)
            )
            if in_c != out_c else nn.Identity()
        )

        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv(x)
        out += self.shortcut(x)
        return self.relu(out)

class StrongCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            ResBlock(1, 32),
            nn.MaxPool2d(2),

            ResBlock(32, 64),
            nn.MaxPool2d(2),  # becomes 8x8

            ResBlock(64, 128),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 12)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = StrongCNN().to(device)
model.load_state_dict(torch.load("./model/model_epoch_190.pt", map_location=device))
model.eval()

print("✔ Model loaded successfully")

input_folder = "./crops_binary_inv"
output_root = "./sorted_digits"
os.makedirs(output_root, exist_ok=True)

for d in range(12):
    os.makedirs(os.path.join(output_root, str(d)), exist_ok=True)

for fname in os.listdir(input_folder):
    if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
        continue

    path = os.path.join(input_folder, fname)
    
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (32, 32))
    img = img.astype(np.float32) / 255.0

    img = torch.tensor(img).unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(img).argmax(dim=1).item()

    shutil.copy(path, os.path.join(output_root, str(pred), fname))

print("\n✔ All predictions completed and images sorted into folders.")


In [ ]:
# crop to thresh images

import cv2
import numpy as np
import glob
import os

input_folder = "./crops/"
binary_output_folder = "./crops_binary_inv/"

os.makedirs(binary_output_folder, exist_ok=True)

lower_red1 = np.array([0, 15, 15])
upper_red1 = np.array([25, 255, 255])

lower_red2 = np.array([155, 15, 15])
upper_red2 = np.array([179, 255, 255])

kernel = np.ones((1,1), np.uint8)

image_path = ["./crops/25-16.jpg", "./crops/25-24.jpg", "./crops/25-7.jpg", "./crops/26-12.jpg", "./crops/26-23.jpg", "./crops/30-6.jpg", "./crops/66-31.jpg", 
              "./crops/40-16.jpg", "./crops/40-29.jpg", "./crops/41-2.jpg", "./crops/41-6.jpg", "./crops/41-19.jpg", "./crops/64-29.jpg", "./crops/58-1.jpg"]

for img_path in glob.glob(input_folder + "/*.jpg"):
# for img_path in image_path:
    img = cv2.imread(img_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 | mask2

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    h, s, v = cv2.split(hsv)

    s = s.astype(np.float32)
    s[mask > 0] *= 4
    s = np.clip(s, 0, 255).astype(np.uint8)

    hsv_enhanced = cv2.merge([h, s, v])
    final_img = cv2.cvtColor(hsv_enhanced, cv2.COLOR_HSV2BGR)

    filename = os.path.basename(img_path)

    gray = cv2.cvtColor(final_img, cv2.COLOR_BGR2GRAY)
    _, binary_inv = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(binary_inv, kernel, iterations=2)
    binary_inv = cv2.resize(binary_inv, (28, 28), interpolation=cv2.INTER_NEAREST)

    cv2.imwrite(os.path.join(binary_output_folder, filename), binary_inv)
    print("Saved:", filename, "and binary_inv version")

In [ ]:
# modal training 5th attempt

import os
import cv2
import torch
import numpy as np
import shutil
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

class ResBlock(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(),
            nn.Conv2d(out_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c)
        )
        self.shortcut = (
            nn.Conv2d(in_c, out_c, 1, bias=False)
            if in_c != out_c else nn.Identity()
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv(x)
        out = out + self.shortcut(x)
        return self.relu(out)

class StrongCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            ResBlock(1, 32),
            nn.MaxPool2d(2),    # 28 -> 14
            ResBlock(32, 64),
            nn.MaxPool2d(2),    # 14 -> 7
            ResBlock(64, 128)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 12)
        )
    def forward(self, x):
        return self.classifier(self.features(x))

model = StrongCNN().to(device)
model.load_state_dict(torch.load("./model/final_28x28_model.pt", map_location=device))
model.eval()
print("✔ Model loaded successfully")

input_folder = "./crops_binary_inv"
output_root  = "./sorted_digits"
os.makedirs(output_root, exist_ok=True)

for d in range(12):
    os.makedirs(os.path.join(output_root, str(d)), exist_ok=True)

for fname in os.listdir(input_folder):
    if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
        continue

    path = os.path.join(input_folder, fname)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = img.astype(np.float32) / 255.0
    img_tensor = torch.tensor(img).unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(img_tensor).argmax(dim=1).item()

    dest = os.path.join(output_root, str(pred), fname)
    shutil.copy(path, dest)

print("\n✔ All images processed & sorted successfully!")

In [ ]:
# image to pdf converter

from PIL import Image, ImageOps
import os

folder_path = "./pdf_based"
output_pdf = "output.pdf"

valid_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".tiff"]

images = []

for file in sorted(os.listdir(folder_path)):
    ext = os.path.splitext(file)[1].lower()
    if ext in valid_extensions:
        img_path = os.path.join(folder_path, file)
        img = Image.open(img_path)

        # Fix EXIF rotation if present
        img = ImageOps.exif_transpose(img)

        img = img.convert("RGB")   # required for PDF
        images.append(img)

if len(images) == 0:
    print("No valid images found!")
else:
    images[0].save(output_pdf, save_all=True, append_images=images[1:])
    print("PDF created:", output_pdf)
